In [82]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [83]:
lines = list(open('annotate_test2.txt', 'r'))
date_list = []
key_list = []
for i in range(len(lines)) :
    l = lines[i].split()[0] + " " + lines[i].split()[1]
    l = l.replace(',','.')
    key_list.append(lines[i].split()[-1])
    date_list.append(dt.datetime.strptime(l, "%Y-%m-%d %H:%M:%S.%f"))

keys = pd.DataFrame(
    {'datetime': date_list,
    'values': key_list})

In [84]:
keys['values'] = keys['values'].str.replace('1','Anger')
keys['values'] = keys['values'].str.replace('2','Joy')
keys['values'] = keys['values'].str.replace('3','Sadness')
keys['values'] = keys['values'].str.replace('4','Fear')
keys['values'] = keys['values'].str.replace('5','Disgust')
keys['values'] = keys['values'].str.replace('6','Surprise')
keys['values'] = keys['values'].str.replace('0','No emotion')

In [85]:
def event_duration(df):

    start = df.iloc[0]
    on_time = []
    off_time = []
    affect = []
    on_time.append(start['datetime'])
    affect.append(start['values'])

    for j in range(len(df.iloc[1:])):
        j += 1
        line = df.iloc[j]
        if j == 1: #starting point
            if line['values'] == start['values']:
                running_value = line['values']
                continue
            else:
                off_time.append(line['datetime'])
                on_time.append(line['datetime'])
                affect.append(line['values'])
                running_value = line['values']

        if line['values'] == running_value:
            continue 
        else: 
            off_time.append(line['datetime'])
            on_time.append(line['datetime'])
            affect.append(line['values'])
            running_value = line['values']

    off_time.append(df['datetime'].iloc[-1])       
    emotions = pd.DataFrame({
        'Task': affect,
        'Start': on_time, 
        'Finish': off_time
    })
    
    return emotions

In [86]:
emotions = event_duration(keys)

In [88]:
emotions

,Task,Start,Finish
0,'Joy',2019-04-01 14:00:03.437,2019-04-01 14:00:12.958
1,'Surprise',2019-04-01 14:00:12.958,2019-04-01 14:00:20.414
2,'Anger',2019-04-01 14:00:20.414,2019-04-01 14:00:24.740
3,'Surprise',2019-04-01 14:00:24.740,2019-04-01 14:01:20.939
4,'Anger',2019-04-01 14:01:20.939,2019-04-01 14:01:27.703
5,'Joy',2019-04-01 14:01:27.703,2019-04-01 14:01:37.774
6,'Anger',2019-04-01 14:01:37.774,2019-04-01 14:01:49.555
7,'Joy',2019-04-01 14:01:49.555,2019-04-01 14:02:59.362
8,'Sadness',2019-04-01 14:02:59.362,2019-04-01 14:03:00.343
9,'Surprise',2019-04-01 14:03:00.343,2019-04-01 14:03:40.943


In [87]:
import plotly.plotly as py 
import plotly.figure_factory as ff
import plotly.graph_objs as go
# from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode()



rgb = []
for c in range(6):
    r = list(np.random.choice(range(256), size=3))
    s2 = ','.join(map(str,r))
    s1 = "rgb("
    s3 = ")"
    rgb.append(s1 + s2 + s3)

fig = ff.create_gantt(emotions, colors=rgb, group_tasks=True)
py.iplot(fig, filename='annotate_test')
